In [10]:
# %reset

<h2>Pre Process Dataset</h2>
<h6>get dataset</br>
split train and test data</br>
spilt every matrix song to one second segment</br></h6>

In [25]:
import numpy as np

DATASET_PATH = "./all.npy"
with open(DATASET_PATH, 'rb') as f:
    data = np.load(f, allow_pickle=True)
data.shape

(13281,)

In [26]:
from sklearn.model_selection import train_test_split

data_train, data_test, _, _ = train_test_split(data, data, test_size=0.2, random_state=42)
del data
data_train.shape, data_test.shape, data_train[0].shape

((10624,), (2657,), (1995, 12))

In [27]:
import librosa
import tensorflow as tf


In [31]:
sample_rate = 16000
hop_length = int(sample_rate * 0.1)
n_fft = int(sample_rate * 0.2)
DIFF = 1
allowed_duration = 10000
special_value = -10
frame_sec_indexes = [librosa.time_to_frames(i, sample_rate, n_fft=n_fft, hop_length=hop_length)
                     for i in range(1, allowed_duration, DIFF)]

max_frames_in_diff=max([frame_sec_indexes[i+1] - frame_sec_indexes[i] for i in range(len(frame_sec_indexes)-1)])

In [41]:
def clean_frame_matrix(feature):

    # add padding
    full_matrix = np.full((max_frames_in_diff, 12), special_value, dtype=np.float32)
    full_matrix[:feature.shape[0], :feature.shape[1]] = feature

    return full_matrix

def split_features(features):
    # TODO change method for better run time

    split_features = np.empty((0, 10, 12))
    # song_indexes = dict()
    for i in range(len(features)):
        # pre_len = len(split_features)
        split_feature = np.split(features[i], [each for each in frame_sec_indexes if each < len(features[i])])
        split_features = np.append(split_features, np.array([clean_frame_matrix(each) for each in split_feature]),
                                   axis=0)
        print(f'\r{i} done', end='\r')

        # add data to song_indexes {song3: (start_index, end_index), }
        # song_indexes[i] = (pre_len, len(split_features))

    return split_features


In [30]:
data_train = split_features(data_train)
data_train.shape

(2415387, 10, 12)

In [42]:
data_test = split_features(data_test)
data_test.shape

(615124, 10, 12)

<h2>seq2seq model</h2>


In [ ]:
from keras.layers import LSTM, GRU, Dense, Input, RepeatVector, TimeDistributed, Masking
from keras.models import Model

tf.keras.backend.clear_session()

encoder_inputs = Input(shape=(None, 12))
masking = Masking(mask_value=special_value)(encoder_inputs)
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(masking)
encoder_states = [state_h, state_c]
# define training decoder
decoder_inputs = RepeatVector(12)(encoder_outputs)

# decoder_inputs = Input(shape=(None, 12))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_time = TimeDistributed(Dense(12, activation='softmax'))
decoder_outputs = decoder_time(decoder_outputs)
model = Model(encoder_inputs, decoder_outputs)
# define inference encoder
encoder_model = Model(encoder_inputs, encoder_outputs)
# define inference decoder
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_inputs_layer = Input(shape=(12, 128))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_layer, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_time(decoder_outputs)
decoder_model = Model([decoder_inputs_layer] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history = model.fit(data_train, data_train, epochs=20)
history